In [21]:
# Разделяем диаметр и толщину стенки и чистим их
def split_d(x):
    x = str(x)
    x = x.replace(',', '.')
    return re.split('х|Х|x|X', x)[0]

def split_s(x):
    x = str(x)
    x = x.replace(',', '.')
    try:
        return re.split('х|Х|x|X', x)[1]
    except:
        return x
    
def clean_d_s(x):
    x = str(x)
    x = x.replace('п', '')
    x = x.replace('П', '')
    x = x.replace('nan', '')
    x = x.replace(',', '.')
    return x

In [22]:
def clean_steps(x):
    x = str(x)
    x = x.replace(u'X', '/')
    x = x.replace(u'x', '/')
    x = x.replace(u'Х', '/')
    x = x.replace(u'х', '/')
    x = x.replace(',', '.')
    x = x.replace('\\', '/')
    x = x.replace('/-', '/0')
    x = x.replace('-/', '0/')
    x = x.replace('***', '0')
    x = x.replace('*', '0')
    x = x.replace('-', '/')
    x = x.replace('nan', '')
    return x

def convert_step0(x):
    return clean_steps(x).split('/')[0]
   
def convert_step1(x):
    try:
        return clean_steps(x).split('/')[1]
    except:
        return x

In [23]:
import pandas as pd
import numpy as np
import re

In [24]:
df = pd.read_excel('DATA/1-8/1-8.xlsx')

In [25]:
df.columns

Index(['№ пакета', '№ партии', 'плавка', 'НД (гост, ту)', 'марка стали',
       'кол-во, шт.', 'масса\nтн', 'D/S', 'время начала т/о',
       'шаг балок закалочная/отпускная печи, сек', '1 зона по ВТР закалка',
       '2 зона по ВТР закалка', '3 зона по ВТР закалка',
       'Скорость прохождения трубы через спрейер, м/с',
       't˚ C трубы после спреера', 'время выдержки в закалочной ванне, сек.',
       't˚ C трубы после ванны', '1 зона ВТР и уставка отпуск',
       '2 зона ВТР и уставка отпуск', '3 зона ВТР и уставка отпуск',
       '4 зона ВТР и уставка отпуск', '5 зона ВТР и уставка отпуск',
       '№ пакета.1', '№ партии.1', 'кол-во, шт..1', 'масса\nтн.1',
       'Unnamed: 26'],
      dtype='object')

In [29]:
df = df.groupby(['№ партии.1','плавка', 'марка стали', 'D/S','шаг балок закалочная/отпускная печи, сек'])['масса\nтн.1'].sum()

In [30]:
df = df.reset_index()

In [32]:
df.shape

(110, 6)

In [33]:
df[u'шаг балок закалочная печь, сек'] = df['шаг балок закалочная/отпускная печи, сек'].apply(convert_step0)
df[u'шаг балок отпускная печь, сек'] = df['шаг балок закалочная/отпускная печи, сек'].apply(convert_step1)
df['диаметр'] = df['D/S'].apply(split_d)
df['толщина стенки'] = df['D/S'].apply(split_s)
df['толщина стенки'] = df['толщина стенки'].apply(clean_d_s)
df['толщина стенки'] = df['толщина стенки'].apply(lambda x: x.replace('107.95', ''))
df['диаметр'] = df['диаметр'].apply(clean_d_s)

In [34]:
del df['шаг балок закалочная/отпускная печи, сек']
del df['D/S']

In [35]:
df.to_excel('jule_econom_8.xlsx')

In [37]:
df = df.groupby(['марка стали','толщина стенки','диаметр', 'шаг балок закалочная печь, сек', 'шаг балок отпускная печь, сек'])['масса\nтн.1'].sum()
df = df.reset_index()
df.to_excel('jule_groups_8.xlsx')